In [3]:
import pandas as pd

nsmc_fname = 'ratings.txt'
nsmc_total_data = pd.read_csv(nsmc_fname, sep='\t')
text_data_list = nsmc_total_data['document'].dropna().values

print(text_data_list[:4])

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ'
 '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.'
 '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.'
 '와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런게 진짜 영화지']


In [58]:
import re
from konlpy.tag import Mecab; mecab=Mecab()
from tqdm import tqdm

corpus = [' '.join([yy for yy, zz in mecab.pos(''.join(re.findall('[가-힣 ]', xx))) if zz[0] not in ('J', 'S', 'U')]) for xx in tqdm(text_data_list)]

100%|██████████| 199992/199992 [00:15<00:00, 13062.34it/s]


In [59]:
from sklearn.feature_extraction.text import CountVectorizer

vect = CountVectorizer(min_df=10, max_df=10000)
x = vect.fit_transform(corpus)

print(x.toarray().shape)
print(len(vect.vocabulary_))
print(vect.vocabulary_.get('희망'))

print(corpus[0])
print(vect.build_analyzer()(corpus[0]))

(199992, 10005)
10005
9974
어릴 때 보 고 지금 다시 봐도 재밌 어요
['어릴', '지금', '다시', '봐도', '재밌', '어요']


In [60]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(min_df=10, max_df=10000).fit(corpus)
print(tf_idf.transform(corpus).toarray().shape)
print(len(tf_idf.vocabulary_))

(199992, 10005)
10005


In [61]:
import operator

df_columns = [xx for xx, _ in sorted(tf_idf.vocabulary_.items(), key=operator.itemgetter(0))]
df_tf_idf = pd.DataFrame(tf_idf.transform(corpus).toarray(), columns=df_columns)
print(df_tf_idf.shape)
df_tf_idf.head()

(199992, 10005)


,가게,가격,가관,가까운,가까움,가까워,가까이,가깝,가끔,가나,...,힐링,힘겹,힘내,힘드,힘든,힘든데,힘들,힘없,힘찬,힙합
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
test_idx_num = 6

print(text_data_list[test_idx_num])
print(corpus[test_idx_num].split())
print([(xx, df_tf_idf.iloc[test_idx_num][xx]) for xx in corpus[test_idx_num].split() if xx in df_columns])

완전 감동입니다 다시봐도 감동
['완전', '감동', '입니다', '다시', '봐도', '감동']
[('완전', 0.37583009691354297), ('감동', 0.6575650012125697), ('입니다', 0.3761915831317115), ('다시', 0.36163180289462304), ('봐도', 0.39250775705750224), ('감동', 0.6575650012125697)]
